In [4]:
from pathlib import Path

In [5]:
if Path.cwd().name == "notebooks":
	%cd ..

In [8]:
import config as cfg

In [11]:
import pandas as pd
pandas_config = [('max_colwidth', 300), ('display.max_columns', 100), ('display.max_colwidth', 300), ('display.max_rows', 500), ('display.float_format', '%.5f')]
# pd.set_option(*(*("max_colwidth", 500), *("display.max_column", 150)))



In [12]:
from itertools import starmap
list(starmap(pd.set_option, pandas_config))

[None, None, None, None]

In [9]:
import pandas as pd
pd.get_option("max_colwidth")

300

In [2]:
import yaml
import os
import dotenv
from pathlib import Path
from pydantic import BaseModel, ValidationError
from types import SimpleNamespace
from typing import Dict, List, Union, NewType
from typing_extensions import TypedDict

# class ConfigModel(BaseModel):
# 	templates: None
# 	path: Dict[str, Union[Path, Dict[str, Union[Path, None]], None]]
PathType = Dict[str, Union[Path, None]]

class ConfigPath(BaseModel):
	path: Dict[str, Union[Path, PathType, None]]

class ConfigPath(BaseModel):
	path: TypedDict("path", )
	
class ConfigModel(BaseModel):
	templates: None
	path: None

class Config:
	def __init__(self) -> None:
		self.config_path = Path().cwd() / 'config'
	
	def _dict_to_paths(self, d: Dict, parent=Path()):
		paths = {}
		for key, value in d.items():
			current_path = parent / key
			if isinstance(value, dict):
				paths[key] = self._dict_to_paths(value, current_path)
			else:
				paths[key] = current_path
		return paths
	
	def _set_paths(self, config: Dict) -> Dict:
		config["path"] = self._dict_to_paths(config["path"])
		return config

	def load_config(self) -> ConfigModel:
		config = {
			path.stem: yaml.safe_load(path.read_text())
			for path in self.config_path.glob('*.yaml')
		}
		return config

/tmp/ipykernel_1264/2269264499.py:19: DeprecationWarning: Failing to pass a value for the 'fields' parameter is deprecated and will be disallowed in Python 3.15. To create a TypedDict class with 0 fields using the functional syntax, pass an empty dictionary, e.g. `path = TypedDict('path', {})`.
  path: TypedDict("path", )


In [4]:
config = Config()
config_dict = config.load_config()
config_dict

{'templates': None,
 'path': {'data': {'base': 'data/',
   'external': 'data/external',
   'interim': 'data/interim',
   'processed': 'data/processed',
   'raw': 'data/raw'},
  'notebooks': 'notebooks/',
  'scripts': 'scripts/'}}

In [195]:
import yaml
import os
import dotenv
from pathlib import Path
from types import SimpleNamespace

In [172]:
def _convert_dict_to_namespace(_dict: Dict) -> SimpleNamespace:
    if isinstance(_dict, dict):
        namespace = SimpleNamespace()
        setattr(namespace, 'config', _dict)
        for key, value in _dict.items():
            if isinstance(value, dict):
                setattr(namespace, key, _convert_dict_to_namespace(value))
            else:
                setattr(namespace, key, value)
        return namespace
    else:
        raise Exception("The input ")
    return _dict

In [6]:
config_path = Path().cwd() / 'config'

def convert_dict_to_namespace(_dict: Dict) -> SimpleNamespace:
    if isinstance(_dict, dict):
        namespace = SimpleNamespace()
        setattr(namespace, 'config', _dict)
        for key, value in _dict.items():
            if isinstance(value, dict):
                setattr(namespace, key, _convert_dict_to_namespace(value))
            else:
                setattr(namespace, key, value)
        return namespace
    else:
        raise Exception("The input must be a dictionary")
    return _dict

def process_paths(paths_namespace, base_path, config_path=None):
    """Recursively process paths, handling only namespace objects"""
    if config_path is None:
        config_path = SimpleNamespace()
    items = [
        (key, getattr(paths_namespace, key))
        for key in dir(paths_namespace)
        if not key.startswith('_') and not callable(getattr(paths_namespace, key))
    ]
    for key, value in items:
        if hasattr(value, '__dict__'):
            nested_namespace = SimpleNamespace()
            setattr(config_path, key, process_paths(value, base_path, nested_namespace))
        elif isinstance(value, str):
            setattr(config_path, key, base_path / value)
        else:
            setattr(config_path, key, value)
    return config_path

def set_paths(config):
    base_path = Path().cwd()
    paths = getattr(config, 'path')  # Use getattr for namespace
    config_path = process_paths(paths, base_path)
    setattr(config, 'path', config_path)  # Use setattr for namespace
    return config


In [7]:
config_dict_n = convert_dict_to_namespace(config_dict)
set_paths(config_dict_n)

namespace(config={'templates': None,
                  'path': {'data': {'base': 'data/',
                    'external': 'data/external',
                    'interim': 'data/interim',
                    'processed': 'data/processed',
                    'raw': 'data/raw'},
                   'notebooks': 'notebooks/',
                   'scripts': 'scripts/'}},
          templates=None,
          path=namespace(config={'data': {'base': 'data/',
                                  'external': 'data/external',
                                  'interim': 'data/interim',
                                  'processed': 'data/processed',
                                  'raw': 'data/raw'},
                                 'notebooks': 'notebooks/',
                                 'scripts': 'scripts/'},
                         data=namespace(base=PosixPath('/home/leobit/Development/Projects/aws-doc-ragqa/research/data'),
                                        config={'base': 'data/',
  

In [10]:
config_dict_n.path.data.external

PosixPath('/home/leobit/Development/Projects/aws-doc-ragqa/research/data/external')

In [11]:
import config as cfg

In [13]:
cfg.path.data.external

PosixPath('/home/leobit/Development/Projects/aws-doc-ragqa/research/data/external')

In [48]:
config_dict = ConfigDict(config="a", num=1)

In [49]:
config_dict.config

'a'

In [150]:
from typing_extensions import TypedDict
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = 'Jane Doe'
    stats: TypedDict("Stats", {"age": int, "height": float})

In [151]:
user.id

NameError: name 'user' is not defined

In [152]:
# Standard library imports
import pathlib
from typing import Union

# Third party library imports
from pydantic import BaseModel, field_validator

cwd = Path().cwd() / 'config'
config_file = cwd / "config.yml"


class App(BaseModel):
    """App config class."""
    name: str
    version: str


class Testing(BaseModel):
    """Testing config class."""
    path: pathlib.Path

    @field_validator('path')
    @classmethod
    def transform_path(cls, path: Union[pathlib.Path, str]) -> pathlib.Path:
        """Transforms a relative path into an absolute path."""
        
        if not isinstance(path, (pathlib.Path | str)):
            message = "Error: Path must be pathlib.Path or str."
            raise TypeError(message)

        path = pathlib.Path(path)

        if path.is_absolute():
            return path

        return cwd / path


class Config(BaseModel):
    """CLI config class."""
    app: App
    testing: Testing


def _load_yml_config(path: pathlib.Path):
    """Classmethod returns YAML config"""
    try:
        return yaml.safe_load(path.read_text())

    except FileNotFoundError as error:
        message = "Error: yml config file not found."
        logger.exception(message)
        raise FileNotFoundError(error, message) from error


Settings = Config(**_load_yml_config(config_file))

In [155]:
Settings.testing.path

PosixPath('/home/leobit/Development/Projects/aws-doc-ragqa/research/config/example_date.txt')

In [230]:
import config as cfg

In [233]:
cfg.path.data.base

PosixPath('/home/leobit/Development/Projects/aws-doc-ragqa/research/data')

In [4]:
import weave

help(weave.Evaluation)

Help on class Evaluation in module weave.flow.eval:

class Evaluation(weave.flow.obj.Object)
 |  Evaluation(*, name: Optional[str] = None, description: Optional[str] = None, ref: Optional[weave.trace.refs.ObjectRef] = None, dataset: typing.Annotated[weave.flow.dataset.Dataset, BeforeValidator(func=<function cast_to_dataset at 0x7eff61655f80>, json_schema_input_type=PydanticUndefined)], scorers: Optional[list[Annotated[Union[weave.trace.op.Op, weave.flow.scorer.Scorer], BeforeValidator(func=<function cast_to_scorer at 0x7eff61656020>, json_schema_input_type=PydanticUndefined)]]] = None, preprocess_model_input: Optional[Callable[[dict], dict]] = None, trials: int = 1, evaluation_name: Union[str, Callable[[weave.trace.weave_client.Call], str], NoneType] = None) -> None
 |  
 |  Sets up an evaluation which includes a set of scorers and a dataset.
 |  
 |  Calling evaluation.evaluate(model) will pass in rows from a dataset into a model matching
 |      the names of the columns of the datase

In [1]:

from llama_index.core.llama_dataset import download_llama_dataset
from llama_index.core.llama_pack import download_llama_pack
from llama_index.core import VectorStoreIndex

# download and install dependencies for benchmark dataset
rag_dataset, documents = download_llama_dataset(
    "EvaluatingLlmSurveyPaperDataset", "./data"
)

In [7]:
reference_contexts = rag_dataset.to_pandas().iloc[0].reference_contexts
for rc in reference_contexts:
    print(rc)
    print("-" * 100)
    print()

Evaluating Large Language Models: A
Comprehensive Survey
Zishan Guo∗, Renren Jin∗, Chuang Liu∗, Yufei Huang, Dan Shi, Supryadi
Linhao Yu, Yan Liu, Jiaxuan Li, Bojian Xiong, Deyi Xiong†
Tianjin University
{guozishan, rrjin, liuc_09, yuki_731, shidan, supryadi}@tju.edu.cn
{linhaoyu, yan_liu, jiaxuanlee, xbj1355, dyxiong}@tju.edu.cn
Abstract
Large language models (LLMs) have demonstrated remarkable capabilities
across a broad spectrum of tasks. They have attracted significant attention
and been deployed in numerous downstream applications. Nevertheless, akin
to a double-edged sword, LLMs also present potential risks. They could
suffer from private data leaks or yield inappropriate, harmful, or misleading
content. Additionally, the rapid progress of LLMs raises concerns about the
potential emergence of superintelligent systems without adequate safeguards.
To effectively capitalize on LLM capacities as well as ensure their safe and
beneficial development, it is critical to conduct a rigorou

In [3]:
from dataclasses import asdict, dataclass

@dataclass
class X:
    i: int = 2

x = X(i=42)
x.i

42

In [ ]:
model_cfg: float = getattr(cfg.app, model_provider)
if hasattr(model_cfg, "similarity_top_k"):
	similarity_top_k = model_cfg.similarity_top_k
if hasattr(model_cfg, "temperature"):
	temperature = model_cfg.temperature
if hasattr(model_cfg, "context_size"):
	context_size = model_cfg.context_size

In [7]:
%cd ..

/home/leobit/Development/Projects/aws-doc-ragqa/research


In [8]:
import config as cfg

In [9]:
model_provider = "gemini"
model_cfg = getattr(cfg.app, model_provider)

In [11]:
if hasattr(model_cfg, "similarity_top_k"):
    print("yes")

yes
